In [1]:
#Librerías de análisis de datos:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import pickle

from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier

from pickle import dump
from pickle import load

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, precision_score, roc_auc_score

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import sys
!{sys.executable} -m pip install xgboost lightgbm catboost

In [3]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [4]:
# Buenas prácticas.
path_to_data = "./stroke_dataset.csv"
data_stroke = pd.read_csv(path_to_data)

In [5]:
#Cantidad de entradas y columnas del dataset.
data_stroke.shape

(4981, 11)

In [6]:
data_stroke.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [7]:
data_stroke.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4981 entries, 0 to 4980
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             4981 non-null   object 
 1   age                4981 non-null   float64
 2   hypertension       4981 non-null   int64  
 3   heart_disease      4981 non-null   int64  
 4   ever_married       4981 non-null   object 
 5   work_type          4981 non-null   object 
 6   Residence_type     4981 non-null   object 
 7   avg_glucose_level  4981 non-null   float64
 8   bmi                4981 non-null   float64
 9   smoking_status     4981 non-null   object 
 10  stroke             4981 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 428.2+ KB


In [8]:
data_stroke.nunique()

gender                  2
age                   104
hypertension            2
heart_disease           2
ever_married            2
work_type               4
Residence_type          2
avg_glucose_level    3895
bmi                   342
smoking_status          4
stroke                  2
dtype: int64

In [9]:
data_stroke.isnull().sum()

gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [10]:
categoricas=["gender", "ever_married","work_type","Residence_type","smoking_status"]
numericas=["age", "hypertension","heart_disease","avg_glucose_level","bmi","stroke"]

In [11]:
data_stroke.describe()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,4981.000000,4981.000000,4981.000000,4981.000000,4981.000000,4981.000000
mean,43.419859,0.096165,0.055210,105.943562,28.498173,0.049789
std,22.662755,0.294848,0.228412,45.075373,6.790464,0.217531
min,0.080000,0.000000,0.000000,55.120000,14.000000,0.000000
25%,25.000000,0.000000,0.000000,77.230000,23.700000,0.000000
50%,45.000000,0.000000,0.000000,91.850000,28.100000,0.000000
75%,61.000000,0.000000,0.000000,113.860000,32.600000,0.000000
max,82.000000,1.000000,1.000000,271.740000,48.900000,1.000000


In [12]:
data_stroke.duplicated().sum()

0

## Preprocesamiento: 

## Preprocesamiento: 

def onehot_encode(df,column):
    df = df.copy()
    
    dummies = pd.get_dummies( df [column],prefix=column)
    df = pd.concat([df,dummies], axis=1)
    df = df.drop(column,axis=1)
    
    return df 
    

X_train, X_test, y_train, y_test = preprocess_inputs(data_stroke)

#Hacemos función que hace una copia como primer paso

def preprocess_inputs(df):
    df = df.copy()

#2.- Si quicieramos eliminar la columna 
    # df = df.drop("id", axis=1)
    
#3.- Despues de identificar las clases  dentro de cada columna, trabajamos los binarios
    df["ever_married"] = df["ever_married"].replace({"No":0,"Yes":1})
    df["gender"] = df["gender"].replace({"Male":0,"Female":1})
    df["Residence_type"] = df["Residence_type"].replace({"Urban":0,"Rural":1})
    
#4.-Haciendo Onehot_encode
    for column in ["work_type","smoking_status"]:
        df = onehot_encode(df,column = column)
        
        
#5.- Separamos la columna stroke (variable a predecir) 
    y = df['stroke']
    X = df.drop('stroke', axis=1)
    
#6.- Definiendo Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
#7.- Escalamos las variables Xtest, Xtrain  
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    
   
    return X_train, X_test, y_train, y_test

y_train 

X_train.var()

AQUI COMIENZA EL MEOLLO DEL ASUNTO

{'transformer_numerica': slice(0, 5, None),
 'transformer_categorica': slice(5, 19, None),
 'remainder': slice(0, 0, None)}

## ENTRENAMOS


Comenzamos a entrenar nuestros modelos con los datos desbalanceados, para ir en orden y que se vea la mejora del modelo, con los balanceos.

In [18]:
models = {
    "                   Logistic Regression": LogisticRegression(),
    "                   K-Nearest Neighbors": KNeighborsClassifier(),
    "                         Decision Tree": DecisionTreeClassifier(),
    "Support Vector Machine (Linear Kernel)": LinearSVC(),
    "   Support Vector Machine (RBF Kernel)": SVC(),
    "                        Neural Network": MLPClassifier(),
    "                         Random Forest": RandomForestClassifier(),
    "                     Gradient Boosting": GradientBoostingClassifier(),
    "                               XGBoost": XGBClassifier(eval_metric='mlogloss'),
    "                              LightGBM": LGBMClassifier(),
    "                              CatBoost": CatBoostClassifier(verbose=0)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")



                   Logistic Regression trained.
                   K-Nearest Neighbors trained.
                         Decision Tree trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                        Neural Network trained.
                         Random Forest trained.
                     Gradient Boosting trained.
                               XGBoost trained.
                              LightGBM trained.
                              CatBoost trained.


In [19]:
y_train.value_counts()

0    3315
1     171
Name: stroke, dtype: int64

In [20]:
print("Model Performance\n-----------------")
for name, model in models.items():
    y_pred = model.predict(X_test)
    
    y_pred_train = model.predict(X_train)
    
    print(
        "\n" + name + " Accuracy: {:.3f}%\n\t\t\t\t       F1-Score: {:.5f}\n\t\t\t\t              Recall: {:.5f}\n\t\t\t\t       Precision: {:.5f}\n\t\t\t\t       Roc_Au: {:.5f}\n\t\t\t\t"\
        .format(accuracy_score(y_test, y_pred) * 100, f1_score(y_test, y_pred),recall_score(y_test, y_pred),precision_score(y_test, y_pred),roc_auc_score(y_test, y_pred) ))
    
    print( confusion_matrix(y_test, y_pred))
    
    print(
        "\n" + name + " Accuracy_train: {:.3f}%\n\t\t\t\t       F1-Score_train: {:.5f}\n\t\t\t\t              Recall_Train: {:.5f}\n\t\t\t\t       Precision_Train: {:.5f}\n\t\t\t\t       Roc_Au_Train: {:.5f}\n\t\t\t\t"\
        .format(accuracy_score(y_train, y_pred_train) * 100, f1_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train) ))
    #Hemos quitado la matriz de confusion
    #,recall_score(y_train, y_pred),precision_score(y_train, y_pred)
    print(confusion_matrix(y_train, y_pred_train))
    
    print(
    np.abs(((((accuracy_score(y_train, y_pred_train))-accuracy_score(y_test, y_pred))/(accuracy_score(y_test, y_pred)) *100)))
       )
           

Model Performance
-----------------

                   Logistic Regression Accuracy: 94.916%
				       F1-Score: 0.02564
				              Recall: 0.01299
				       Precision: 1.00000
				       Roc_Au: 0.50649
				
[[1418    0]
 [  76    1]]

                   Logistic Regression Accuracy_train: 95.095%
				       F1-Score_train: 0.00000
				              Recall_Train: 0.00000
				       Precision_Train: 0.00000
				       Roc_Au_Train: 0.50000
				
[[3315    0]
 [ 171    0]]
0.18782469048650188

                   K-Nearest Neighbors Accuracy: 94.716%
				       F1-Score: 0.04819
				              Recall: 0.02597
				       Precision: 0.33333
				       Roc_Au: 0.51158
				
[[1414    4]
 [  75    2]]

                   K-Nearest Neighbors Accuracy_train: 95.209%
				       F1-Score_train: 0.07735
				              Recall_Train: 0.04094
				       Precision_Train: 0.70000
				       Roc_Au_Train: 0.52002
				
[[3312    3]
 [ 164    7]]
0.5212334406228691

                        

In [21]:
print("Model Performance\n-----------------")
for name, model in models.items():
    y_pred = model.predict(X_test)
    
    print(
        "\n" + name + " Accuracy: {:.3f}%\n\t\t\t\t       F1-Score: {:.5f}"\
        .format(accuracy_score(y_test, y_pred) * 100, f1_score(y_test, y_pred)
           ))
    #Hemos quitado la matriz de confusion
    #,recall_score(y_train, y_pred),precision_score(y_train, y_pred)
           

Model Performance
-----------------

                   Logistic Regression Accuracy: 94.916%
				       F1-Score: 0.02564

                   K-Nearest Neighbors Accuracy: 94.716%
				       F1-Score: 0.04819

                         Decision Tree Accuracy: 92.107%
				       F1-Score: 0.18056

Support Vector Machine (Linear Kernel) Accuracy: 94.849%
				       F1-Score: 0.00000

   Support Vector Machine (RBF Kernel) Accuracy: 94.849%
				       F1-Score: 0.00000

                        Neural Network Accuracy: 94.582%
				       F1-Score: 0.04706

                         Random Forest Accuracy: 94.849%
				       F1-Score: 0.00000

                     Gradient Boosting Accuracy: 94.983%
				       F1-Score: 0.07407

                               XGBoost Accuracy: 93.913%
				       F1-Score: 0.04211

                              LightGBM Accuracy: 93.980%
				       F1-Score: 0.00000

                              CatBoost Accuracy: 94.716%
				       F1-Score: 0.04819


## OVERSAMPLING


In [22]:
y_train.value_counts()

0    3315
1     171
Name: stroke, dtype: int64

In [23]:
oversampled_data = pd.concat([X_train, y_train], axis=1).copy()

num_samples = y_train.value_counts()[0] - y_train.value_counts()[1]
new_samples = oversampled_data.query("stroke == 1").sample(num_samples, replace=True, random_state=1)

oversampled_data = pd.concat([oversampled_data, new_samples], axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

##Cambios segur Rb
y_oversampled = oversampled_data['stroke']
X_oversampled = oversampled_data.drop('stroke', axis=1)

#Def Train-Test-Split

X_oversampled_train, X_oversampled_test, y_oversampled_train, y_oversampled_test = train_test_split(X_oversampled, y_oversampled, train_size=0.7, shuffle=True, random_state=1)

In [24]:
X_oversampled_train.var()

gender                            1.011813
age                               0.938879
hypertension                      1.711155
heart_disease                     1.998487
ever_married                      0.795716
Residence_type                    0.998616
avg_glucose_level                 1.413906
bmi                               0.870330
work_type_Govt_job                1.034610
work_type_Private                 0.999041
work_type_Self-employed           1.244830
work_type_children                0.592118
smoking_status_Unknown            0.891125
smoking_status_formerly smoked    1.212737
smoking_status_never smoked       0.977086
smoking_status_smokes             1.097818
dtype: float64

In [25]:
y_oversampled_train

3253    1
4972    0
2408    0
6594    1
2761    0
       ..
905     1
5192    0
3980    0
235     0
5157    0
Name: stroke, Length: 4641, dtype: int64

In [26]:
oversampled_data 

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,work_type_Govt_job,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,stroke
0,0.860964,0.760070,-0.329816,4.153470,0.709390,1.009801,0.301358,1.210494,2.5208,-1.147292,-0.437479,-0.395245,-0.660113,-0.465568,-0.763614,2.380208,1
1,0.860964,-0.296505,-0.329816,-0.240763,0.709390,-0.990294,-0.192419,-1.199677,-0.3967,0.871618,-0.437479,-0.395245,-0.660113,-0.465568,1.309562,-0.420131,0
2,0.860964,1.200310,-0.329816,-0.240763,0.709390,-0.990294,1.957183,0.813698,-0.3967,-1.147292,2.285826,-0.395245,-0.660113,2.147912,-0.763614,-0.420131,1
3,-1.161489,0.451902,-0.329816,-0.240763,-1.409662,1.009801,-0.005382,-0.170945,-0.3967,0.871618,-0.437479,-0.395245,-0.660113,2.147912,-0.763614,-0.420131,0
4,0.860964,1.508478,-0.329816,-0.240763,0.709390,1.009801,-1.014280,-0.215034,-0.3967,-1.147292,2.285826,-0.395245,-0.660113,2.147912,-0.763614,-0.420131,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6625,0.860964,0.319830,-0.329816,-0.240763,0.709390,1.009801,-0.209362,-1.038019,-0.3967,-1.147292,2.285826,-0.395245,-0.660113,-0.465568,1.309562,-0.420131,0
6626,-1.161489,1.068238,-0.329816,-0.240763,0.709390,-0.990294,-0.636906,-0.156249,-0.3967,-1.147292,2.285826,-0.395245,-0.660113,-0.465568,-0.763614,2.380208,1
6627,-1.161489,1.332382,-0.329816,-0.240763,0.709390,-0.990294,-0.743847,0.402205,-0.3967,0.871618,-0.437479,-0.395245,-0.660113,-0.465568,-0.763614,2.380208,1
6628,-1.161489,-0.428577,-0.329816,-0.240763,-1.409662,-0.990294,0.037526,0.269940,-0.3967,0.871618,-0.437479,-0.395245,-0.660113,-0.465568,-0.763614,2.380208,0


In [27]:
y_oversampled_train.value_counts()

1    2322
0    2319
Name: stroke, dtype: int64

In [28]:
## Entrenando los modelos con la data balanceada
models = {
    "                   Logistic Regression": LogisticRegression(),
    "                   K-Nearest Neighbors": KNeighborsClassifier(),
    "                         Decision Tree": DecisionTreeClassifier(),
    "Support Vector Machine (Linear Kernel)": LinearSVC(),
    "   Support Vector Machine (RBF Kernel)": SVC(),
    "                        Neural Network": MLPClassifier(),
    "                         Random Forest": RandomForestClassifier(),
    "                     Gradient Boosting": GradientBoostingClassifier(),
    "                               XGBoost": XGBClassifier(eval_metric='mlogloss'),
    "                              LightGBM": LGBMClassifier(),
    "                              CatBoost": CatBoostClassifier(verbose=0)
}


for name, model in models.items():
    model.fit(X_oversampled_train, y_oversampled_train)
    print(name + " Entrenado.")


                   Logistic Regression Entrenado.
                   K-Nearest Neighbors Entrenado.
                         Decision Tree Entrenado.
Support Vector Machine (Linear Kernel) Entrenado.
   Support Vector Machine (RBF Kernel) Entrenado.
                        Neural Network Entrenado.
                         Random Forest Entrenado.
                     Gradient Boosting Entrenado.
                               XGBoost Entrenado.
                              LightGBM Entrenado.
                              CatBoost Entrenado.


In [29]:
y_oversampled_train.value_counts()

1    2322
0    2319
Name: stroke, dtype: int64

In [30]:
print("Model Performance\n-----------------")
for name, model in models.items():
    y_oversampled_pred = model.predict(X_oversampled_test)
    
    y_oversampled_pred_train = model.predict(X_oversampled_train)


    print(
        "\n" + name + " Accuracy: {:.3f}%\n\t\t\t\t       F1-Score: {:.5f}\n\t\t\t              Recall: {:.5f}\n\t\t\t\t       Precision: {:.5f}\n\t\t\t\t       Roc_Au: {:.5f}\n\t\t\t\t"\
        .format(accuracy_score(y_test, y_pred) * 100, f1_score(y_test, y_pred),recall_score(y_test, y_pred),precision_score(y_test, y_pred),roc_auc_score(y_test, y_pred) ))
    
    print( confusion_matrix(y_test, y_pred))
    
    print(
        "\n" + name + " Accuracy_train: {:.3f}%\n\t\t\t\t       F1-Score_train: {:.5f}\n\t\t\t\t        Recall_Train: {:.5f}\n\t\t\t\t       Precision_Train: {:.5f}\n\t\t\t\t       Roc_Au_Train: {:.5f}\n\t\t\t\t"\
        .format(accuracy_score(y_train, y_pred_train) * 100, f1_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train) ))
   # print(
   # np.abs((((accuracy_score(y_test, y_pred))/(accuracy_score(y_train, y_pred_train)) *100)))
    #      ) Confusion_Matrix: {:.5f}\n\t\t\t\t Confusion_Matrix_Train: {:.5f}\n\t\t\t\t,confusion_matrix(y_test,y_pred),
    
    
    print( confusion_matrix(y_train, y_pred_train))
    
    print(
    "\n" + name + " Overfitting:\n\t\t\t\t",
    np.abs(((((accuracy_score(y_train, y_pred_train))-accuracy_score(y_test, y_pred))/(accuracy_score(y_test, y_pred)) *100)))
       )

Model Performance
-----------------

                   Logistic Regression Accuracy: 94.716%
				       F1-Score: 0.04819
			              Recall: 0.02597
				       Precision: 0.33333
				       Roc_Au: 0.51158
				
[[1414    4]
 [  75    2]]

                   Logistic Regression Accuracy_train: 97.676%
				       F1-Score_train: 0.68966
				        Recall_Train: 0.52632
				       Precision_Train: 1.00000
				       Roc_Au_Train: 0.76316
				
[[3315    0]
 [  81   90]]

                   Logistic Regression Overfitting:
				 3.1258812489668135

                   K-Nearest Neighbors Accuracy: 94.716%
				       F1-Score: 0.04819
			              Recall: 0.02597
				       Precision: 0.33333
				       Roc_Au: 0.51158
				
[[1414    4]
 [  75    2]]

                   K-Nearest Neighbors Accuracy_train: 97.676%
				       F1-Score_train: 0.68966
				        Recall_Train: 0.52632
				       Precision_Train: 1.00000
				       Roc_Au_Train: 0.76316
				
[[3315    0]
 [  81   90]]



In [31]:
######Test con valores oversampled#####
print("Model Performance\n-----------------")
for name, model in models.items():
    y_oversampled_pred = model.predict(X_oversampled_test)
    
    y_oversampled_pred_train = model.predict(X_oversampled_train)


    print(
        "\n" + name + " Accuracy: {:.3f}%\n\t\t\t\t       F1-Score: {:.5f}\n\t\t\t              Recall: {:.5f}\n\t\t\t\t       Precision: {:.5f}\n\t\t\t\t       Roc_Au: {:.5f}\n\t\t\t\t"\
        .format(accuracy_score(y_oversampled_test, y_oversampled_pred) * 100, f1_score(y_oversampled_test, y_oversampled_pred),recall_score(y_oversampled_test, y_oversampled_pred),precision_score(y_oversampled_test, y_oversampled_pred),roc_auc_score(y_oversampled_test, y_oversampled_pred) ))
    
    print( confusion_matrix(y_oversampled_test, y_oversampled_pred))
    
    print(
        "\n" + name + " Accuracy_train: {:.3f}%\n\t\t\t\t       F1-Score_train: {:.5f}\n\t\t\t\t        Recall_Train: {:.5f}\n\t\t\t\t       Precision_Train: {:.5f}\n\t\t\t\t       Roc_Au_Train: {:.5f}\n\t\t\t\t"\
        .format(accuracy_score(y_oversampled_train, y_oversampled_pred_train) * 100, f1_score(y_oversampled_train, y_oversampled_pred_train),recall_score(y_oversampled_train, y_oversampled_pred_train),precision_score(y_oversampled_train, y_oversampled_pred_train),roc_auc_score(y_oversampled_train, y_oversampled_pred_train) ))
   # print(
   # np.abs((((accuracy_score(y_test, y_pred))/(accuracy_score(y_train, y_pred_train)) *100)))
    #      ) Confusion_Matrix: {:.5f}\n\t\t\t\t Confusion_Matrix_Train: {:.5f}\n\t\t\t\t,confusion_matrix(y_test,y_pred),
    
    
    print( confusion_matrix(y_oversampled_train, y_oversampled_pred_train))
    
    print(
    "\n" + name + " Overfitting:\n\t\t\t\t",
    np.abs(((((accuracy_score(y_oversampled_train, y_oversampled_pred_train))-accuracy_score(y_oversampled_test, y_oversampled_pred))/(accuracy_score(y_oversampled_test, y_oversampled_pred)) *100)))
       )

Model Performance
-----------------

                   Logistic Regression Accuracy: 78.482%
				       F1-Score: 0.79383
			              Recall: 0.82981
				       Precision: 0.76085
				       Roc_Au: 0.78488
				
[[737 259]
 [169 824]]

                   Logistic Regression Accuracy_train: 77.591%
				       F1-Score_train: 0.78819
				        Recall_Train: 0.83333
				       Precision_Train: 0.74768
				       Roc_Au_Train: 0.77587
				
[[1666  653]
 [ 387 1935]]

                   Logistic Regression Overfitting:
				 1.1348036972636548

                   K-Nearest Neighbors Accuracy: 90.447%
				       F1-Score: 0.91268
			              Recall: 1.00000
				       Precision: 0.83939
				       Roc_Au: 0.90462
				
[[806 190]
 [  0 993]]

                   K-Nearest Neighbors Accuracy_train: 94.441%
				       F1-Score_train: 0.94737
				        Recall_Train: 1.00000
				       Precision_Train: 0.90000
				       Roc_Au_Train: 0.94437
				
[[2061  258]
 [   0 2322]]

        

In [ ]:
#Para usar GridSearch

from sklearn.model_selection import GridSearchCV


#Creando la Gridsearch




In [49]:
##Intento de 2da tecnica de balanceo

data_stroke

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
4976,Male,41.0,0,0,No,Private,Rural,70.15,29.8,formerly smoked,0
4977,Male,40.0,0,0,Yes,Private,Urban,191.15,31.1,smokes,0
4978,Female,45.0,1,0,Yes,Govt_job,Rural,95.02,31.8,smokes,0
4979,Male,40.0,0,0,Yes,Private,Rural,83.94,30.0,smokes,0


In [50]:
## Aqui sacamos las X y y para procesarlas fuera de la funcion 
## Probando método nuevo
#Eliminamos Transported de X porque es el valor a predecir. La X es mayúscula
y = data_stroke['stroke']
X = data_stroke.drop('stroke', axis=1)

In [52]:
X.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked


In [51]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: stroke, dtype: int64

In [53]:
categoricas=["gender", "ever_married","work_type","Residence_type","smoking_status"]
numericas=["age", "hypertension","heart_disease","avg_glucose_level","bmi"]

In [54]:
#Importanción de Sklearn
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

In [55]:
#Creamos transformers para cada categoría de variables.
transformer_numerico = ("transformer_numerica", MinMaxScaler(), numericas)
transformer_categorico = ("transformer_categorica", OneHotEncoder(), categoricas)
transformer = ColumnTransformer([transformer_numerico, transformer_categorico], remainder="passthrough")

In [56]:
X = transformer.fit_transform(X)

In [72]:
transformer_final= pd.DataFrame(X, columns = transformer.get_feature_names_out())

In [73]:
transformer_final

,transformer_numerica__age,transformer_numerica__hypertension,transformer_numerica__heart_disease,transformer_numerica__avg_glucose_level,transformer_numerica__bmi,transformer_categorica__gender_Female,transformer_categorica__gender_Male,transformer_categorica__ever_married_No,transformer_categorica__ever_married_Yes,transformer_categorica__work_type_Govt_job,transformer_categorica__work_type_Private,transformer_categorica__work_type_Self-employed,transformer_categorica__work_type_children,transformer_categorica__Residence_type_Rural,transformer_categorica__Residence_type_Urban,transformer_categorica__smoking_status_Unknown,transformer_categorica__smoking_status_formerly smoked,transformer_categorica__smoking_status_never smoked,transformer_categorica__smoking_status_smokes
0,0.816895,0.0,1.0,0.801265,0.647564,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.975586,0.0,1.0,0.234512,0.530086,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.597168,0.0,0.0,0.536008,0.584527,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.963379,1.0,0.0,0.549349,0.286533,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.987793,0.0,0.0,0.605161,0.429799,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4976,0.499512,0.0,0.0,0.069384,0.452722,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4977,0.487305,0.0,0.0,0.627966,0.489971,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4978,0.548340,1.0,0.0,0.184194,0.510029,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4979,0.487305,0.0,0.0,0.133044,0.458453,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [58]:
transformer.output_indices_

{'transformer_numerica': slice(0, 5, None),
 'transformer_categorica': slice(5, 19, None),
 'remainder': slice(0, 0, None)}

In [79]:
#Cargando el archivo de transformer a pickle para subirlo a streamlit

file_trans = open('transformer_entrenado.pkl', 'wb')
pickle.dump(transformer_final, file_trans)

In [59]:
positivo = y.sum()

In [60]:
negativo = y.shape[0]-positivo

In [61]:
print("Res Positivos: ", positivo, "Res Negativos:", negativo)

Res Positivos:  248 Res Negativos: 4733


In [62]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [63]:
from imblearn.combine import SMOTETomek

smoteT = SMOTETomek()

In [64]:
X_smoteT, y_smoteT = smoteT.fit_resample(X, y)
Positivo_smoteT = y_smoteT.sum()
Negativo_smoteT = y_smoteT.shape[0]-Positivo_smoteT


print("Res Positivas:",Positivo_smoteT,",Res Negativas:", Negativo_smoteT)
X_smoteT 

X_smoteT_train, X_smoteT_test, y_smoteT_train, y_smoteT_test = train_test_split(X_smoteT, y_smoteT, train_size=0.7, shuffle=True, random_state=1)


Res Positivas: 4674 ,Res Negativas: 4674


In [65]:
## Corriendo los modelos con balanceo SMOTETomek
models = {
    "                   Logistic Regression": LogisticRegression(),
    "                   K-Nearest Neighbors": KNeighborsClassifier(),
    "                         Decision Tree": DecisionTreeClassifier(),
    "Support Vector Machine (Linear Kernel)": LinearSVC(),
    "   Support Vector Machine (RBF Kernel)": SVC(),
    "                        Neural Network": MLPClassifier(),
    "                         Random Forest": RandomForestClassifier(),
    "                     Gradient Boosting": GradientBoostingClassifier(),
    "                               XGBoost": XGBClassifier(eval_metric='mlogloss'),
    "                              LightGBM": LGBMClassifier(),
    "                              CatBoost": CatBoostClassifier(verbose=0)
}


for name, model in models.items():
    model.fit(X_smoteT_train, y_smoteT_train)
    print(name + " Entrenado.")


                   Logistic Regression Entrenado.
                   K-Nearest Neighbors Entrenado.
                         Decision Tree Entrenado.
Support Vector Machine (Linear Kernel) Entrenado.
   Support Vector Machine (RBF Kernel) Entrenado.
                        Neural Network Entrenado.
                         Random Forest Entrenado.
                     Gradient Boosting Entrenado.
                               XGBoost Entrenado.
                              LightGBM Entrenado.
                              CatBoost Entrenado.


In [66]:
 y_smoteT_train.value_counts()

1    3282
0    3261
Name: stroke, dtype: int64

In [69]:
######Test con valores smoteT#####
print("Model Performance\n-----------------")
for name, model in models.items():
    y_smoteT_pred = model.predict(X_smoteT_test)
    
    y_smoteT_pred_train = model.predict(X_smoteT_train)


    print(
        "\n" + name + " Accuracy: {:.3f}%\n\t\t\t\t       F1-Score: {:.5f}\n\t\t\t              Recall: {:.5f}\n\t\t\t\t       Precision: {:.5f}\n\t\t\t\t       Roc_Au: {:.5f}\n\t\t\t\t"\
        .format(accuracy_score(y_smoteT_test, y_smoteT_pred) * 100, f1_score(y_smoteT_test, y_smoteT_pred),recall_score(y_smoteT_test, y_smoteT_pred),precision_score(y_smoteT_test, y_smoteT_pred),roc_auc_score(y_smoteT_test, y_smoteT_pred) ))
    
    print( confusion_matrix(y_smoteT_test, y_smoteT_pred))
    
    print(
        "\n" + name + " Accuracy_train: {:.3f}%\n\t\t\t\t       F1-Score_train: {:.5f}\n\t\t\t\t        Recall_Train: {:.5f}\n\t\t\t\t       Precision_Train: {:.5f}\n\t\t\t\t       Roc_Au_Train: {:.5f}\n\t\t\t\t"\
        .format(accuracy_score(y_smoteT_train, y_smoteT_pred_train) * 100, f1_score(y_smoteT_train, y_smoteT_pred_train),recall_score(y_smoteT_train, y_smoteT_pred_train),precision_score(y_smoteT_train, y_smoteT_pred_train),roc_auc_score(y_smoteT_train, y_smoteT_pred_train) ))
   # print(
   # np.abs((((accuracy_score(y_test, y_pred))/(accuracy_score(y_train, y_pred_train)) *100)))
    #      ) Confusion_Matrix: {:.5f}\n\t\t\t\t Confusion_Matrix_Train: {:.5f}\n\t\t\t\t,confusion_matrix(y_test,y_pred),
    
    
    print( confusion_matrix(y_smoteT_train, y_smoteT_pred_train))
    
    print(
    "\n" + name + " Overfitting accuracy:\n\t\t\t\t",
    np.abs(((((accuracy_score(y_smoteT_train, y_smoteT_pred_train))-accuracy_score(y_smoteT_test, y_smoteT_pred))/(accuracy_score(y_smoteT_test, y_smoteT_pred)) *100)))
       )
    
    print(
    "\n" + name + " Overfitting recall:\n\t\t\t\t",
    np.abs(((((recall_score(y_smoteT_train, y_smoteT_pred_train))-recall_score(y_smoteT_test, y_smoteT_pred))/(recall_score(y_smoteT_test, y_smoteT_pred)) *100)))
       )
        
    print(
    "\n" + name + " Overfitting presicion:\n\t\t\t\t",
    np.abs(((((precision_score(y_smoteT_train, y_smoteT_pred_train))-precision_score(y_smoteT_test, y_smoteT_pred))/(precision_score(y_smoteT_test, y_smoteT_pred)) *100)))
       )

Model Performance
-----------------

                   Logistic Regression Accuracy: 78.788%
				       F1-Score: 0.80027
			              Recall: 0.85632
				       Precision: 0.75110
				       Roc_Au: 0.78839
				
[[1018  395]
 [ 200 1192]]

                   Logistic Regression Accuracy_train: 79.933%
				       F1-Score_train: 0.80913
				        Recall_Train: 0.84796
				       Precision_Train: 0.77370
				       Roc_Au_Train: 0.79917
				
[[2447  814]
 [ 499 2783]]

                   Logistic Regression Overfitting accuracy:
				 1.4531090184460254

                   Logistic Regression Overfitting recall:
				 0.9766511662147428

                   Logistic Regression Overfitting presicion:
				 3.0085893725755697

                   K-Nearest Neighbors Accuracy: 89.091%
				       F1-Score: 0.89914
			              Recall: 0.97989
				       Precision: 0.83069
				       Roc_Au: 0.89157
				
[[1135  278]
 [  28 1364]]

                   K-Nearest Neighbors Accuracy_train:

In [70]:
modelo_elegido = XGBClassifier()
modelo_elegido.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [71]:
modelo_pickle = 'XGBClassifier.plk'
pickle.dump(modelo_elegido, open(modelo_pickle, 'wb'))